In [17]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb
import datetime
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, classification_report
pd.set_option('display.max_columns', None)

In [18]:
df=pd.read_csv('DATA.csv')
df=df.dropna(axis=0)

In [18]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF']]

In [19]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength',  
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF', 'ratio', 'source_ecoli']]

In [27]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF', 'ratio', 'source_ecoli']]

In [3]:
y=df['EColi.']
X=df.loc[:, ['up_down', 'curvature', 'inclination', 'tilt_direction', 'altitude', 'disto_river', 'disto_stations', 
             'disto_mainroad', 'disto_syorizyo', 'supply_hours', 'no_water_days', 'total_population ', 'population_served',
             'popu-served', 'number_taps', 'pipelength', 'pipelength_per_pipe', 'served/pipes', '(popu-served)/pipes', 
             'oldest_pipe_age', 'ST', 'RSF', 'FL', 'PF', 'RF', 'ratio', 'source_ecoli']]

In [28]:
X.shape

(218, 27)

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3,
                                                  random_state=1, stratify=y)
print('Labels counts in y:', np.bincount(y))
print('Lables counts in y_train:', np.bincount(y_train))
print('Lables counts in y_test:', np.bincount(y_test))

Labels counts in y: [ 98 120]
Lables counts in y_train: [68 84]
Lables counts in y_test: [30 36]


In [23]:
X_train_std=(X_train-X_train.mean(axis=0))/X_train.std(axis=0)
X_test_std=(X_test-X_train.mean(axis=0))/X_train.std(axis=0)

In [24]:
def threshold(fare):
    if fare<=0:
        return 0
    else:
        return 1

In [25]:
X_train_std['ST']=X_train_std['ST'].apply(threshold)
X_train_std['RSF']=X_train_std['RSF'].apply(threshold)
X_train_std['FL']=X_train_std['FL'].apply(threshold)
X_train_std['PF']=X_train_std['PF'].apply(threshold)
X_train_std['RF']=X_train_std['RF'].apply(threshold)
X_test_std['ST']=X_test_std['ST'].apply(threshold)
X_test_std['RSF']=X_test_std['RSF'].apply(threshold)
X_test_std['FL']=X_test_std['FL'].apply(threshold)
X_test_std['PF']=X_test_std['PF'].apply(threshold)
X_test_std['RF']=X_test_std['RF'].apply(threshold)

# SVC

In [26]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
                                      
dic={}
for i in range(len(param_range)):
    svm=SVC(kernel='linear', C=param_range[i], random_state=1)
    svm.fit(X_train_std, y_train)
    score1 = svm.score(X_test_std, y_test)
    score2 = svm.score(X_train_std, y_train)
    dic[svm]=score1

    if score1>max_score:
        val=cross_val_score(estimator=svm, X=X_train_std, y=y_train, cv=3)
        val1=np.mean(val)
        print('params: %s, train: %3f, test: %3f, val: %3f' %(svm, score2, score1, val1))

params: SVC(C=0.0001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: SVC(C=0.001, kernel='linear', random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: SVC(C=0.01, kernel='linear', random_state=1), train: 0.585526, test: 0.560606, val: 0.559216
params: SVC(C=0.1, kernel='linear', random_state=1), train: 0.644737, test: 0.606061, val: 0.605621
params: SVC(kernel='linear', random_state=1), train: 0.684211, test: 0.681818, val: 0.573203
params: SVC(C=10.0, kernel='linear', random_state=1), train: 0.690789, test: 0.681818, val: 0.539739
params: SVC(C=100.0, kernel='linear', random_state=1), train: 0.697368, test: 0.575758, val: 0.566536
params: SVC(C=1000.0, kernel='linear', random_state=1), train: 0.690789, test: 0.606061, val: 0.559869


# ロジスティック回帰

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0
SearchMethod = 0
param_range=[0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
                                      
dic={}
for i in range(len(param_range)):
    lr=LogisticRegression(C=param_range[i], random_state=1)
    lr.fit(X_train_std, y_train)
    score1 = lr.score(X_test_std, y_test)
    score2 = lr.score(X_train_std, y_train)
    dic[lr]=score1

    if score1>max_score:
        val=cross_val_score(estimator=lr, X=X_train_std, y=y_train, cv=3)
        val1=np.mean(val)
        print('params: %s, train: %3f, test: %3f, val: %3f' %(lr, score2, score1, val1))

params: LogisticRegression(C=0.0001, random_state=1), train: 0.552632, test: 0.545455, val: 0.552680
params: LogisticRegression(C=0.001, random_state=1), train: 0.552632, test: 0.530303, val: 0.552680
params: LogisticRegression(C=0.01, random_state=1), train: 0.611842, test: 0.560606, val: 0.572418
params: LogisticRegression(C=0.1, random_state=1), train: 0.638158, test: 0.575758, val: 0.592418
params: LogisticRegression(random_state=1), train: 0.690789, test: 0.560606, val: 0.572680


C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

params: LogisticRegression(C=10.0, random_state=1), train: 0.690789, test: 0.636364, val: 0.500915
params: LogisticRegression(C=100.0, random_state=1), train: 0.703947, test: 0.696970, val: 0.527190
params: LogisticRegression(C=1000.0, random_state=1), train: 0.717105, test: 0.696970, val: 0.540392


C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\kurokiso\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logist

# random forest

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.70
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    dic[forest]=score1

                    if score1>max_score:
                        val=cross_val_score(estimator=forest,
                                            X=X_train_std, y=y_train, cv=3)
                        val1=np.mean(val)
                        print('params: %s, score: %3f, val: %3f' %(forest, score1, val1))



params: RandomForestClassifier(max_depth=8, n_estimators=12, random_state=1), score: 0.727273, val: 0.540261
params: RandomForestClassifier(max_depth=10, n_estimators=12, random_state=1), score: 0.727273, val: 0.533856
params: RandomForestClassifier(max_depth=10, n_estimators=13, random_state=1), score: 0.712121, val: 0.533987
params: RandomForestClassifier(max_depth=11, n_estimators=13, random_state=1), score: 0.712121, val: 0.553725
params: RandomForestClassifier(max_depth=12, n_estimators=13, random_state=1), score: 0.727273, val: 0.540523
params: RandomForestClassifier(max_depth=8, n_estimators=14, random_state=1), score: 0.712121, val: 0.546797
params: RandomForestClassifier(max_depth=12, n_estimators=19, random_state=1), score: 0.712121, val: 0.533725
params: RandomForestClassifier(max_depth=13, n_estimators=19, random_state=1), score: 0.712121, val: 0.520523
params: RandomForestClassifier(max_depth=7, n_estimators=20, random_state=1), score: 0.712121, val: 0.540392
params: Rando

KeyboardInterrupt: 

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.75
SearchMethod = 0
RFC_grid ={"n_estimators": [i for i in range(1, 51)],
           "criterion": ["gini", "entropy"],
           "max_depth":[i for i in range(1, 21)]
          }
           
                                      
dic={}
for i in range(len(RFC_grid['n_estimators'])):
           for l in range(len(RFC_grid["criterion"])):
               for k in range(len(RFC_grid["max_depth"])):
                    forest=RandomForestClassifier(max_depth=RFC_grid["max_depth"][k], criterion=RFC_grid["criterion"][l], n_estimators=RFC_grid['n_estimators'][i], random_state=1)
                    forest.fit(X_train_std, y_train)
                    score1 = forest.score(X_test_std, y_test)
                    score2 = forest.score(X_train_std, y_train)
                    dic[forest]=score1

                    if score1>max_score:
                        val=cross_val_score(estimator=forest,
                                            X=X_train_std, y=y_train,
                                            cv=3)
                        val1=np.mean(val)
                        print('params: %s, train: %3f, test: %3f, val: %3f' %(forest, score2, score1, val1))

params: RandomForestClassifier(criterion='entropy', max_depth=14, n_estimators=23,
                       random_state=1), train: 1.000000, test: 0.757576, val: 0.540261


# Boosted forest

In [15]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

max_score =0.65
SearchMethod = 0
RFC_grid ={"base_estimator" : [DecisionTreeClassifier(max_depth=x) 
                                  for x in range(5, 10)],
              "learning_rate" : [0.5, 1.0, 1.5],
              "n_estimators": [i for i in range(1, 100)],
           }   
                                      
dic={}
for i in range(len(RFC_grid['base_estimator'])):
           for l in range(len(RFC_grid["learning_rate"])):
               for k in range(len(RFC_grid["n_estimators"])):
                    ada = AdaBoostClassifier(base_estimator=RFC_grid["base_estimator"][i], learning_rate=RFC_grid["learning_rate"][l], n_estimators=RFC_grid['n_estimators'][k], random_state=1)
                    ada.fit(X_train_std, y_train)
                    score1 = ada.score(X_test_std, y_test)
                    score2 = ada.score(X_train_std, y_train)
                    dic[ada]=score1

                    if score1>max_score:
                        val=cross_val_score(estimator=ada,
                                            X=X_train_std, y=y_train,
                                            cv=3)
                        val1=np.mean(val)
                        print('params: %s, train: %3f, test: %3f, val: %3f' %(ada, score2, score1, val1))


params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=7, random_state=1), train: 1.000000, test: 0.666667, val: 0.526536
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=14, random_state=1), train: 1.000000, test: 0.651515, val: 0.546536
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=17, random_state=1), train: 1.000000, test: 0.651515, val: 0.540000
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=18, random_state=1), train: 1.000000, test: 0.651515, val: 0.559869
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=22, random_state=1), train: 1.000000, test: 0.651515, val: 0.519869
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estima

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=38, random_state=1), train: 1.000000, test: 0.651515, val: 0.552941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=46, random_state=1), train: 1.000000, test: 0.651515, val: 0.553072
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=47, random_state=1), train: 1.000000, test: 0.651515, val: 0.546536
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=48, random_state=1), train: 1.000000, test: 0.651515, val: 0.552941
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, random_state=1), train: 1.000000, test: 0.666667, val: 0.539869
params: AdaBoostClassifier(

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=91, random_state=1), train: 1.000000, test: 0.666667, val: 0.520131
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=92, random_state=1), train: 1.000000, test: 0.666667, val: 0.526667
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=93, random_state=1), train: 1.000000, test: 0.666667, val: 0.520131
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=94, random_state=1), train: 1.000000, test: 0.651515, val: 0.520131
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=0.5, n_estimators=95, random_state=1), train: 1.000000, test: 0.666667, val: 0.520131
params: Ad

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=1.5, n_estimators=94, random_state=1), train: 1.000000, test: 0.651515, val: 0.507582
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=1.5, n_estimators=95, random_state=1), train: 1.000000, test: 0.651515, val: 0.487843
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=1.5, n_estimators=96, random_state=1), train: 1.000000, test: 0.651515, val: 0.487843
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=1.5, n_estimators=97, random_state=1), train: 1.000000, test: 0.651515, val: 0.487843
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=6),
                   learning_rate=1.5, n_estimators=98, random_state=1), train: 1.000000, test: 0.651515, val: 0.494379
params: Ad

params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                   learning_rate=1.5, n_estimators=58, random_state=1), train: 1.000000, test: 0.651515, val: 0.527059
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                   learning_rate=1.5, n_estimators=59, random_state=1), train: 1.000000, test: 0.681818, val: 0.533725
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                   learning_rate=1.5, n_estimators=60, random_state=1), train: 1.000000, test: 0.681818, val: 0.527059
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                   learning_rate=1.5, n_estimators=61, random_state=1), train: 1.000000, test: 0.666667, val: 0.520523
params: AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7),
                   learning_rate=1.5, n_estimators=62, random_state=1), train: 1.000000, test: 0.666667, val: 0.520523
params: Ad